# Code for training CNN on MNIST and saving weights


In [19]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
#initialization code required to make tensorflow work on my systemabs
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
#disabling eager execution 
tf.compat.v1.disable_eager_execution()
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Tensorflow version: ",tf.__version__)

Num GPUs Available:  1
Tensorflow version:  2.4.0


In [109]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.utils import to_categorical
from art.utils import load_mnist

### I am using a simple covnet from the official ART for keras mnist tutorial linked [here](https://github.com/Trusted-AI/adversarial-robustness-toolbox/blob/main/examples/get_started_keras.py)

Using ART's load_mnist() to load MNIST dataset

In [111]:
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

In [112]:
x_test.shape

(10000, 28, 28, 1)

Define and compile the model architecture: taken directly from official ART tutorial linked above

In [113]:
#define
model = Sequential()
model.add(Conv2D(filters=4, kernel_size=(5, 5), strides=1, activation="relu", input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=10, kernel_size=(5, 5), strides=1, activation="relu", input_shape=(23, 23, 4)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dense(10, activation="softmax"))


In [114]:
#compile: Categorical Crossentropy Loss Function and Adam Optimizer used
model.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=0.01), metrics=["accuracy"])

Training model: I use more epochs and a larget batch size than the tutorial

In [120]:

batch_size = 128
epochs = 5
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)


Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 2s 39us/sample - loss: 0.0435 - accuracy: 0.9866 - val_loss: 0.0682 - val_accuracy: 0.9837
Epoch 2/5
54000/54000 [==============================] - 2s 39us/sample - loss: 0.0471 - accuracy: 0.9850 - val_loss: 0.0611 - val_accuracy: 0.9848
Epoch 3/5
54000/54000 [==============================] - 2s 39us/sample - loss: 0.0373 - accuracy: 0.9882 - val_loss: 0.0543 - val_accuracy: 0.9877
Epoch 4/5
54000/54000 [==============================] - 2s 40us/sample - loss: 0.0463 - accuracy: 0.9870 - val_loss: 0.0819 - val_accuracy: 0.9820
Epoch 5/5
54000/54000 [==============================] - 2s 39us/sample - loss: 0.0408 - accuracy: 0.9880 - val_loss: 0.0703 - val_accuracy: 0.9848


Lets sanity check by running inference on the train and test set

In [125]:
loss,accuracy = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Test loss: 0.07105395802160493
Test accuracy: 0.9836


Everything looks good!

Lets save the model weights and create a standard attack testing set for target t=7

In [126]:
#save model weights
model.save_weights("mnist_model_weights.h5")




In [136]:
#save full model for use with FSGM attack
model.save(r'softmax_model')

INFO:tensorflow:Assets written to: softmax_model\assets


In [123]:
#define target class
t = 7

In [127]:
#save standardized test set: 100 examples that are not 7:
#first pull out 
y_test_without_target = y_test[np.argmax(y_test,axis=1)!=t]
x_test_without_target = x_test[np.argmax(y_test,axis=1)!=t]

In [128]:
x_test_without_target.shape

(8972, 28, 28, 1)

In [129]:
y_test_without_target.shape

(8972, 10)

In [130]:
#seeding to standardize across runs
np.random.seed(1) 
#now, shuffle in unison randomly and save top 100 examples 
shuffler = np.random.permutation(len(y_test_without_target))
x_attack_test = x_test_without_target[shuffler][:100]
y_attack_test = y_test_without_target[shuffler][:100]

In [131]:
x_attack_test.shape

(100, 28, 28, 1)

In [132]:
y_attack_test.shape

(100, 10)

In [133]:
np.save(r'x_attack_test',x_attack_test)

In [134]:
np.save(r'y_attack_test', y_attack_test)

In [135]:
#sanity checking accuracy on benign attack samples
test_loss,test_accuracy = model.evaluate(x_attack_test, y_attack_test, verbose=0)
print("Benign Sample Test loss:", test_loss)
print("Benign Sample Test accuracy:", test_accuracy)

Benign Sample Test loss: 0.003133925013210046
Benign Sample Test accuracy: 1.0
